# Spark Basics

In [1]:
import pyspark
import pandas as pd 

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark1 = SparkSession.builder.appName('Saiyan').getOrCreate()

23/07/02 17:24:46 WARN Utils: Your hostname, alexender-ROG-Strix-G512LI-G512LI resolves to a loopback address: 127.0.1.1; using 192.168.1.7 instead (on interface wlo1)
23/07/02 17:24:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/02 17:24:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark1.version

'3.3.1'

In [5]:
# df_spark = spark1.read.csv("Telco_customer_churn.csv", header=True, inferSchema=True)
df = spark1.read.json("people.json")

In [6]:
# df_spark.show(5)
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.columns

['age', 'name']

In [9]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [10]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [11]:
from pyspark.sql.types import StructField, StringType, StructType, IntegerType

data_schema = [StructField('age', IntegerType(), True),
                                StructField('name', StringType(), True)]

In [12]:
final_struc = StructType(fields=data_schema)

In [13]:
df = spark1.read.json('people.json', schema=final_struc)

In [14]:
df.show()
df.printSchema()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [15]:
type(df['age'])

pyspark.sql.column.Column

In [16]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [17]:
a = df.head(2)

In [18]:
a[0].asDict()

{'age': None, 'name': 'Michael'}

In [19]:
df_alter = spark1.createDataFrame(a)
df_alter.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
+----+-------+



In [20]:
# There is no inplace in pyspark DataFrame we have to reassign the value to the existing df or to a new df 
df_new = df.withColumn('new_age', df['age']*2)

In [21]:
df.show()
df_new.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     60|
|  19| Justin|     38|
+----+-------+-------+



In [22]:
# We can write query in pyspark but we need to register the df as table 
df.createOrReplaceTempView('people')

In [23]:
results = spark1.sql("select * from people where age>18")
results.show()

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+

